# "Comparing Apples to Apples"
> "A first look at computer vision with a support vector classifier model."

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [image processing, computer vision, classification]
- hide: false
- search_exclude: true

## Theory
### What is computer vision?
Computer vision is teaching your computer to understand and classify the information in images and videos.

For this post, I will review how to code a simple algorithm that differentiates between pictures of **apples and oranges**.

We will be using the open source computer vision (**OpenCV**) library, the [most popular computer vision library](https://hub.packtpub.com/top-10-computer-vision-tools/).

## Examples
### What do we use computer vision for?
Examples adapted from [this blog post](https://machinelearningmastery.com/what-is-computer-vision/).
- Optical character recognition (OCR)
- Retail (e.g. automated checkouts)
- Medical imaging
- Merging computer-generated imagery (CGI) with live actors in movies
- Surveillance
- Fingerprint recognition and biometrics

## Code
### How to train a simple computer vision model.
### *Download Photographs*
Start by downloading pictures of apples and oranges and putting them into separate folders called `Apples` and `Oranges`.

Ideally you should have [at least 50 photos](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html).

Possible sources:

- Apples (8 varieties) and oranges (1 variety) from the [Fruits 360 Kaggle competition dataset](https://www.kaggle.com/moltean/fruits).
- [Google Images](https://www.google.com/imghp?hl=en) search.
- Search through the [ImageNet database](http://image-net.org/).

### *Import Libraries*
Import the libraries we will need:

In [1]:
import cv2
import numpy as np
import os
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

- We will use the [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) for this model, which will draw a barrier (hyperplane) to distinctively classify the features of images of oranges from the features of images of apples.

### *Convert the Images*
This function generates an array from a folder of fruit photos, as well as a list of corresponding labels. We then use this code on the <mark>Oranges</mark> and <mark>Apples</mark> subfolders.

In [2]:
def fruit_listing(directory_path = "Fruit/", fruit_type = "fruit", resolution = (128, 128)):
    fruit_list = []
    directory = directory_path
    for filename in os.listdir(directory):
        if not filename.startswith('.'):
            fruit_list.append(cv2.resize(cv2.imread(f"{directory}{filename}", 1), resolution))
    fruit_labels = []
    for i in range(0,len(fruit_list)):
        fruit_labels.append(fruit_type)
    return fruit_list, fruit_labels

In [3]:
orange_list, orange_labels = fruit_listing(directory_path = "Oranges/", fruit_type = "orange")
apple_list, apple_labels = fruit_listing(directory_path = "Apples/", fruit_type = "apple")

Hidden files starting with `.` were excluded. (My computer automatically generated the folder attribute file `.DS_Store` in both the `Oranges` and `Apples` folders, but this file is clearly neither an orange nor an apple.)

The files are all resized to `128 x 128`. A higher resolution would slow down the modelling but may result in a more accurate model.

Please note that my photos were located in subdirectories of my Juypter notebook's directory. If yours aren't, then you'll have to type out the whole `directory_path` when calling the function.

### *Create X and y (Features and Target):*

In [4]:
fruit_array = np.stack(orange_list + apple_list, axis = 3)
fruit_label_array = np.array(orange_labels + apple_labels)

In [5]:
X = fruit_array
y = fruit_label_array

X = X.reshape(X.shape[0] * X.shape[1] * X.shape[2], X.shape[3]).T
y = y.reshape(y.shape[0],)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)

The data must be reshaped to turn the feature array from 3-dimensional (pixel x-coordinate, pixel y-coordinate, and pixel color) into 1-dimensional (pixel features). The lines for reshaping the feature array are from [this blog post](https://blog.hyperiondev.com/index.php/2019/02/18/machine-learning/).

### *Train Model and Check Accuracy*

In [6]:
svc = SVC()
svc.fit(X_train, y_train)
preds = svc.predict(X_test)
print("Accuracy:", accuracy_score(y_test, preds))

Accuracy: 0.9090909090909091


My accuracy was 91% with the base support vector classifier. Tuning the hyperparameters could get you an even better result, but I'll leave that part out since hyperparameter tuning is not the focus of this blog post.

### *Check Model on New Test Data*

In [7]:
def fruit_prediction(filename = "test.jpg", resolution = (128, 128)):
    test_fruit = cv2.resize(cv2.imread(f"{filename}", 1), resolution)
    test_fruit = test_fruit.reshape(test_fruit.shape[0] * test_fruit.shape[1] * test_fruit.shape[2], 1).T
    return svc.predict(test_fruit)[0]

In [8]:
fruit_prediction(filename = "test-orange.jpg")

'orange'

In [9]:
fruit_prediction(filename = "test-apple.jpg")

'apple'

The model correctly classified the test orange and test apple!

Please note that my test orange and test apple were located in my Juypter notebook's main directory. If yours aren't, then you'll have to change the function above to access their actual paths.

Now you're ready for the next step, [image classification with a convolutional neural network](https://www.tensorflow.org/tutorials/images/classification).